# Joining DataFrames

## Concatenation

Concatenation is really just pasting records onto a table. You are just adding records, either horizontally or vertically. In Pandas, we do this with the `concat()` function.

### By Column

To add records vertically, to append by row, we use `axis=0`. This is the default setting for `concat()`

In [3]:
import pandas as pd

In [4]:
pip install pandas 

In [5]:
q1  = pd.DataFrame({"Month": ['Jan', 'Feb', 'Mar'],
                    "Sales": [1200, 1000, 1100]})

q2  = pd.DataFrame({"Month": ['April', 'May', 'Jun'],
                    "Sales": [1300, 1400, 1350]})

In [7]:
q1

,Month,Sales
0,Jan,1200
1,Feb,1000
2,Mar,1100


In [9]:
q2

,Month,Sales
0,April,1300
1,May,1400
2,Jun,1350


In [11]:
pd.concat([q1,q2])

,Month,Sales
0,Jan,1200
1,Feb,1000
2,Mar,1100
0,April,1300
1,May,1400
2,Jun,1350


By default, the original row labels are preseved. If we want to dump these and reindex the table, we can pass `ignore_index=True`

In [7]:
#this is a test


In [12]:
pd.concat([q1,q2], ignore_index=True)

,Month,Sales
0,Jan,1200
1,Feb,1000
2,Mar,1100
3,April,1300
4,May,1400
5,Jun,1350


If you want to preserve the keys, and add a key to represent the tables they are coming from, we can pass a list to the`keys` parameter.

In [8]:
pd.concat([q1,q2], keys=['Q1','Q2'])

Month  Sales
Q1 0    Jan   1200
   1    Feb   1000
   2    Mar   1100
Q2 0  April   1300
   1    May   1400
   2    Jun   1350

#### Appending

If you know how to use concetenate properly, you don't need to use append. Append is just a special case of concatenation, where `axis=0, join='outer'` 

In [14]:
q1.append(q2)

,Month,Sales
0,Jan,1200
1,Feb,1000
2,Mar,1100
0,April,1300
1,May,1400
2,Jun,1350


### By Index

It is very straight-forward to perform this on columns, we just change the axis: `axis=1`. Here, records will add up in accordance with the row label.

In [13]:
brics = {'label':      ['BR','RU','IN','CH','SA'],
         'country':    ['Brazil', 'Russia', 'India', 'China', 'South Africa'],
         'capital':    ['Brasilia', 'Moscow', 'New Dehli', 'Beijing', 'Pretoria'],
         'area':       [8.516, 17.10, 3.286, 9.597, 1.221],
         'population': [200.4, 143.5, 1252, 1357, 52.98]}

brics_df = pd.DataFrame(brics).set_index('label')
brics_df

,country,capital,area,population
label,,,,
BR,Brazil,Brasilia,8.516,200.40
RU,Russia,Moscow,17.100,143.50
IN,India,New Dehli,3.286,1252.00
CH,China,Beijing,9.597,1357.00
SA,South Africa,Pretoria,1.221,52.98


In [14]:
gdp = {'label': ['BR','RU','IN','CH','SA'], 'GDP': [1.84, 1.7, 2.87, 14.34, 0.35]}

gdp_df = pd.DataFrame(gdp).set_index('label')
gdp_df

,GDP
label,
BR,1.84
RU,1.70
IN,2.87
CH,14.34
SA,0.35


In [18]:
pd.concat([brics_df, gdp_df], axis=1)

,country,capital,area,population,GDP
label,,,,,
BR,Brazil,Brasilia,8.516,200.40,1.84
RU,Russia,Moscow,17.100,143.50,1.70
IN,India,New Dehli,3.286,1252.00,2.87
CH,China,Beijing,9.597,1357.00,14.34
SA,South Africa,Pretoria,1.221,52.98,0.35


## Merging

Here I will define two new DataFrames: one for all my staff and the cities they work in; and one mapping the cities to countries.

In [19]:
staff  = pd.DataFrame({"Staff": ['Mary', 'John', 'Jack', 'Sally','Mark', 'Jane','Olivia','Pat','Ed'],
                    "City": ['Dublin', 'Berlin', 'Warsaw', 'Paris','Dublin', 'Lisbon','Cork','Dublin','London']})
staff

,Staff,City
0,Mary,Dublin
1,John,Berlin
2,Jack,Warsaw
3,Sally,Paris
4,Mark,Dublin
5,Jane,Lisbon
6,Olivia,Cork
7,Pat,Dublin
8,Ed,London


In [31]:
salaries = pd.DataFrame({'City': ['Paris','Warsaw','Berlin','Dublin', 'London','Limerick'],
                        'Avg Salary':[58000, 45000, 55000, 60000, 62000, 68000]})
salaries

,City,Avg Salary
0,Paris,58000
1,Warsaw,45000
2,Berlin,55000
3,Dublin,60000
4,London,62000
5,Limerick,68000


In this example, our key is going to be `City`. That means, both DataFrames are going to be joined where their values for `City` are matching. In this example, it is fine for my `Staff` table to have the same value for `City` multiple times; however, it must be unique in my `Country` table. That means that my `Country` table is my primary table and `City` here is my primary key. Consequently, my `Staff` table is my foreign or child table and the `City` column in that is my foreign key.

#### Inner Join

For an inner join, only matching records will be returned. You can specify this with `how='inner'`, but it is the default join type, so we can leave it out.

In [33]:
staff.merge(salaries)

,Staff,City,Avg Salary
0,Mary,Dublin,60000
1,Mark,Dublin,60000
2,Pat,Dublin,60000
3,John,Berlin,55000
4,Jack,Warsaw,45000
5,Sally,Paris,58000
6,Ed,London,62000


Notice we have no record for Jane (staff) or for the UK (country), as these failed to find a matching record in the `City` column.

##### Specifying the Columns

It has recognised the common column in both, `City`, and used this as a key for the join. If there wasn't a common column in both, we would have to specify which column to join on. In this example, one of the columns is renamed to `Cities`.

In [34]:
salaries = pd.DataFrame({'Cities': ['Paris','Warsaw','Berlin','Dublin', 'London'],
                        'Avg Salary':[58000, 45000, 55000, 60000, 62000]})
salaries

,Cities,Avg Salary
0,Paris,58000
1,Warsaw,45000
2,Berlin,55000
3,Dublin,60000
4,London,62000


Here we must specify the column in both the left and right table to join on, using the `left_on` and `right_on` parameters. Remember, left is the first table and right is the second.

In [26]:
staff.merge(salaries, left_on='City', right_on='Cities')

,Staff,City,Cities,Avg Salary
0,Mary,Dublin,Dublin,60000
1,Mark,Dublin,Dublin,60000
2,Pat,Dublin,Dublin,60000
3,John,Berlin,Berlin,55000
4,Jack,Warsaw,Warsaw,45000
5,Sally,Paris,Paris,58000


#### Outer Join

An outer join, will return all records in both tables, regardless of whether a match was found or not. We specify this by setting `how='outer'`. (I will reset my Staff table from `Cities` to `City`)

In [35]:
salaries = pd.DataFrame({'City': ['Paris','Warsaw','Berlin','Dublin', 'London'],
                        'Avg Salary':[58000, 45000, 55000, 60000, 62000]})

In [36]:
staff.merge(salaries, how='outer')

,Staff,City,Avg Salary
0,Mary,Dublin,60000.0
1,Mark,Dublin,60000.0
2,Pat,Dublin,60000.0
3,John,Berlin,55000.0
4,Jack,Warsaw,45000.0
5,Sally,Paris,58000.0
6,Jane,Lisbon,NaN
7,Olivia,Cork,NaN
8,Ed,London,62000.0


We now have returned every record from both tables, regardless of whether it made a match. Notice Jane is in the merged table, but has no value for Country; while London is included but has no staff associated with it.

#### Left or Right Join

If you would like to return all the records for a particular table only, you can specify which table. This is done using the descriptors `left` and `right`. Convention is to label the first table as left and the second one as right.

In our example, it is important to us that we want to return a record for all our staff. We are not interested in returning every record from our `salaries` table. So, in this case, a left join would be perfect.

In [37]:
staff.merge(salaries, how='left')

,Staff,City,Avg Salary
0,Mary,Dublin,60000.0
1,John,Berlin,55000.0
2,Jack,Warsaw,45000.0
3,Sally,Paris,58000.0
4,Mark,Dublin,60000.0
5,Jane,Lisbon,NaN
6,Olivia,Cork,NaN
7,Pat,Dublin,60000.0
8,Ed,London,62000.0


After doing a `left`,`right` or `outer` join, it is often a good idea to filter for `Nulls`. This will help you know what you are missing in your merger. Here we are filtering our DataFrame where `Avg Salary` is `Null`

In [38]:
staff_df = staff.merge(salaries, how='left')

staff_df[staff_df['Avg Salary'].isnull()]

,Staff,City,Avg Salary
5,Jane,Lisbon,NaN
6,Olivia,Cork,NaN


### Duplicates

Earlier, I said that my primary table, `Salaries`, must have unique values for `City`. What I was in effect doing here was declaring that as my primary key. But what if I hadn't insisted on that? What if had both `Staff` and `Salaries` had non-unique values for `City`? We would create duplicates.

In [39]:
salaries = pd.DataFrame({'City': ['Paris','Warsaw','Berlin','Dublin', 'London', 'Paris'],
                        'Avg Salary':[58000, 45000, 55000, 60000, 62000, 63000]})
salaries

,City,Avg Salary
0,Paris,58000
1,Warsaw,45000
2,Berlin,55000
3,Dublin,60000
4,London,62000
5,Paris,63000


In [40]:
staff.merge(salaries)

,Staff,City,Avg Salary
0,Mary,Dublin,60000
1,Mark,Dublin,60000
2,Pat,Dublin,60000
3,John,Berlin,55000
4,Jack,Warsaw,45000
5,Sally,Paris,58000
6,Sally,Paris,63000
7,Ed,London,62000


Here, we can see that the record for `Sally` has been duplicated. Because `City` was not unique in our `Salaries` DataFrame (we had two records for `Paris`), after merging, one record was created for 58k and one for 63k.

## Joining on Multiple Columns

So far, we have been joining on a single columns. That means, records in our primary table can be uniquely identified by that column alone. But this might no always be the case. What if I wanted to include average salaries from Paris, Texas and Dublin, Ohio? In this case we could have multiple records for `City`, but we could use a combination of `City` and `Country` to define our primary key.

In [41]:
staff  = pd.DataFrame({"Staff": ['Mary', 'John', 'Jack', 'Sally','Mark', 'Jane'],
                       "City": ['Dublin', 'Berlin', 'Warsaw', 'Paris','Dublin', 'Lisbon'],
                       "Country": ['Ireland', 'Germany', 'Poland', 'France', 'US','Portugal']})
staff

,Staff,City,Country
0,Mary,Dublin,Ireland
1,John,Berlin,Germany
2,Jack,Warsaw,Poland
3,Sally,Paris,France
4,Mark,Dublin,US
5,Jane,Lisbon,Portugal


In [42]:
salaries = pd.DataFrame({'City': ['Paris','Warsaw','Berlin','Dublin', 'London', 'Paris', 'Dublin'],
                         'Country':['France', 'Poland','Germany','Ireland','London', 'US', 'US'],
                        'Avg Salary':[58000, 45000, 55000, 60000, 62000, 48000, 51000]})
salaries

,City,Country,Avg Salary
0,Paris,France,58000
1,Warsaw,Poland,45000
2,Berlin,Germany,55000
3,Dublin,Ireland,60000
4,London,London,62000
5,Paris,US,48000
6,Dublin,US,51000


By default it will use a combination of similarly-named columns to form the key. Here, it recognises `City` and `Country` columns from both tables and joins on them.

In [43]:
staff.merge(salaries)

,Staff,City,Country,Avg Salary
0,Mary,Dublin,Ireland,60000
1,John,Berlin,Germany,55000
2,Jack,Warsaw,Poland,45000
3,Sally,Paris,France,58000
4,Mark,Dublin,US,51000


Stated explicitly, we would pass in a list in for each key.

In [44]:
staff.merge(salaries, left_on=['City', 'Country'], right_on=['City','Country'])

,Staff,City,Country,Avg Salary
0,Mary,Dublin,Ireland,60000
1,John,Berlin,Germany,55000
2,Jack,Warsaw,Poland,45000
3,Sally,Paris,France,58000
4,Mark,Dublin,US,51000


### Join

Join is a merge based on index. While with a merge you can specify the column or index to merge on, join merges both DataFrames on the index. Let's take a look at our `brics` dataset again.

In [45]:
brics_df

,country,capital,area,population
label,,,,
BR,Brazil,Brasilia,8.516,200.40
RU,Russia,Moscow,17.100,143.50
IN,India,New Dehli,3.286,1252.00
CH,China,Beijing,9.597,1357.00
SA,South Africa,Pretoria,1.221,52.98


In [46]:
sales = {'label': ['BR','RU','IN','CH','SA'],
         'sales':  [8000, 17000, 3000, 9000, 10000]}

sales_df = pd.DataFrame(sales).set_index('label')
sales_df

,sales
label,
BR,8000
RU,17000
IN,3000
CH,9000
SA,10000


If we want to `merge()` on the index here (`label`), we would have to state that explicitly using `left_index=True` and `right_index=True`

In [47]:
brics_df.merge(sales_df, left_index=True, right_index=True)

,country,capital,area,population,sales
label,,,,,
BR,Brazil,Brasilia,8.516,200.40,8000
RU,Russia,Moscow,17.100,143.50,17000
IN,India,New Dehli,3.286,1252.00,3000
CH,China,Beijing,9.597,1357.00,9000
SA,South Africa,Pretoria,1.221,52.98,10000


This can be a bit cumbersome, for such a fairly commmon join type, so the `join()` function was created. It is basically just a `merge()` but with `left_index=True`, `right_index=True` and `how='inner'`.

In [48]:
brics_df.join(sales_df)

,country,capital,area,population,sales
label,,,,,
BR,Brazil,Brasilia,8.516,200.40,8000
RU,Russia,Moscow,17.100,143.50,17000
IN,India,New Dehli,3.286,1252.00,3000
CH,China,Beijing,9.597,1357.00,9000
SA,South Africa,Pretoria,1.221,52.98,10000


### Creating a Primary Key by Grouping

So far we have been using datasets where at least one table had a primary key, i.e. the records of the key were unique. This allowed up to merge without fear of create duplication. But what if this is not the case? Then, we can consider grouping our data to create a unique key. This is akin to creating a pivot table. In this example, I have details on penguins across some Antarctic islands. Nothing about these penguins uniquely identifies them.

In [56]:
import seaborn as sns
mpg = sns.load_dataset('mpg')
mpg

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


In [49]:
pip install seaborn

And now, say I have a collection of temperature readings from these islands. Again, nothing to uniquely identify these records

In [51]:
temps = {'island':['Biscoe','Biscoe','Biscoe','Biscoe',
                   'Dream','Dream','Dream','Dream',
                   'Torgersen','Torgersen','Torgersen','Torgersen',
                   'Anvers','Anvers','Anvers','Anvers'],
         'temp':[-2, -3, -5, -4, -5, -1, -3, 0,
                 -5, -4, -4, -5, -3, -1, 0, -5]}

temps = pd.DataFrame(temps)
temps

,island,temp
0,Biscoe,-2
1,Biscoe,-3
2,Biscoe,-5
3,Biscoe,-4
4,Dream,-5
5,Dream,-1
6,Dream,-3
7,Dream,0
8,Torgersen,-5
9,Torgersen,-4


If I wanted to join these two tables together by `island`, it would cause a lot of duplication.

In [52]:
penguins.merge(temps).shape  # Making that join and printing the shape

(1376, 8)

I joined my `penguins` table (`344` rows) with my `temps` table (`16` rows) and I've ended up with `1376` rows!
In this case, what we are going to want to do is first group the data by `island`, making this a unique record (effectively creating a primary key) and then do my merger. 

We group using the `groupby()` method. This is a two step process: We must first provide the column or columns to group by. Then specify how we would like the rows to be aggregated.

In [64]:
# Group by Island, average the numeric columns
mpg_grouped = mpg.groupby(['cylinders']).mean()
mpg_grouped

,mpg,displacement,horsepower,weight,acceleration,model_year
cylinders,,,,,,
3,20.550000,72.500000,99.250000,2398.500000,13.250000,75.500000
4,29.286765,109.796569,78.281407,2308.127451,16.601471,77.073529
5,27.366667,145.000000,82.333333,3103.333333,18.633333,79.000000
6,19.985714,218.142857,101.506024,3198.226190,16.263095,75.928571
8,14.963107,345.009709,158.300971,4114.718447,12.955340,73.902913


We would aggregate each column differently using the `agg()` and passing in a dictionary.

In [77]:
mpg_grouped = mpg.groupby(['cylinders']).agg({'mpg':'mean', 'weight':'sum'})
mpg_grouped

,mpg,weight
cylinders,,
3,20.550000,9594
4,29.286765,470858
5,27.366667,9310
6,19.985714,268651
8,14.963107,423816


We can also group by multiple columns by passing in a list.

In [68]:
penguins.groupby(['island','sex']).mean()

bill_length_mm  bill_depth_mm  flipper_length_mm  \
island    sex                                                        
Biscoe    Female       43.307500      15.191250         205.687500   
          Male         47.119277      16.597590         213.289157   
Dream     Female       42.296721      17.601639         190.016393   
          Male         46.116129      19.066129         196.306452   
Torgersen Female       37.554167      17.550000         188.291667   
          Male         40.586957      19.391304         194.913043   

                  body_mass_g  
island    sex                  
Biscoe    Female  4319.375000  
          Male    5104.518072  
Dream     Female  3446.311475  
          Male    3987.096774  
Torgersen Female  3395.833333  
          Male    4034.782609

### Grouping and Merging

So, to merge our two tables, without creating duplication, we group them both by island, and then merge on this new index.

In [69]:
temps_grouped = temps.groupby(['island']).mean()
temps_grouped

,temp
island,
Anvers,-2.25
Biscoe,-3.50
Dream,-2.25
Torgersen,-4.50


In [71]:
mpg_grouped = mpg.groupby(['mpg']).mean()
mpg_grouped

,cylinders,displacement,horsepower,weight,acceleration,model_year
mpg,,,,,,
9.0,8.0,304.00,193.00,4732.00,18.500000,70.0
10.0,8.0,333.50,207.50,4495.50,14.500000,70.0
11.0,8.0,374.25,187.00,4419.00,12.375000,72.0
12.0,8.0,394.50,185.00,4786.50,12.083333,72.5
13.0,8.0,353.00,158.35,4254.45,12.935000,73.3
...,...,...,...,...,...,...
43.4,4.0,90.00,48.00,2335.00,23.700000,80.0
44.0,4.0,97.00,52.00,2130.00,24.600000,82.0
44.3,4.0,90.00,48.00,2085.00,21.700000,80.0


In [72]:
penguins_grouped.merge(temps_grouped, left_index=True, right_index=True)

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,temp
island,,,,,
Biscoe,45.257485,15.874850,209.706587,4716.017964,-3.50
Dream,44.167742,18.344355,193.072581,3712.903226,-2.25
Torgersen,38.950980,18.429412,191.196078,3706.372549,-4.50


Now we can see if there's any correlation between these penguin features and the average island temperature.